In [ ]:
# paramètres du SMC
N = 10000
P = 1000
nruns = 5
nprocs = -5

# simulation des données
p_group = 5
p_ext = 5

# setting
n, p_ind, likelihood = setting[:3]
pi_ind = pi_group = setting[3:][0]
step = int(p_ind/p_group)

In [ ]:
# créer le data set
data = (X, y, dict_group, p_ext)

# spécifier la loi a priori
prior = dists.BiLevelPrior(p_group=p_group, p_ind=p_ind, pi_group=pi_group, pi_ind=pi_ind, dict_group=dict_group)

# spécifier la méthode d'approximation de la vraisemblance marginale
if likelihood == "ALA":
    model = bin.BilevelALA(data=data, prior=prior)
elif likelihood == "LA":
    model = bin.BilevelLA(data=data, prior=prior)
else:
    print("likelihood approximation method misspecified")
    break

# spécifier la méthode MCMC
mcmc = ssps.MCMCSequenceWF(mcmc=bin.BiLevelBinaryMetropolis(data=data), len_chain=P)

# spécifier le modèle SMC 
smc = ssps.AdaptiveTempering(model=model, move=mcmc, len_chain=P)

# lancer le SMC
output = particles.multiSMC(fk=smc, N=N // P, nruns=nruns, nprocs=nprocs, verbose=0)

# sauvegarde du modèle
filename = open("output_SMC_n_{}_p_{}_lik_{}_prior_{}.pkl".format(n, p_ind, likelihood, pi_ind), "wb")
pickle.dump(output, filename)
filename.close()